In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 21% 5.00M/23.4M [00:00<00:02, 7.16MB/s]
100% 23.4M/23.4M [00:00<00:00, 30.0MB/s]
 34% 9.00M/26.3M [00:00<00:01, 13.4MB/s]
100% 26.3M/26.3M [00:00<00:00, 30.1MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 143MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 139MB/s]


In [3]:
import numpy as np
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [14]:
!unzip /content/train.csv.zip
!unzip /content/test.csv.zip

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [15]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('../content/train.csv').fillna(' ')
test = pd.read_csv('../content/test.csv').fillna(' ')

In [16]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [17]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [18]:
train_text = train['comment_text']

In [19]:
test_text = test['comment_text']

In [20]:
all_text = pd.concat([train_text, test_text])

In [21]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf = True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1,1),
    max_features=10000)
word_vectorizer.fit(all_text)

TfidfVectorizer(max_features=10000, stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')

In [22]:
train_word_features = word_vectorizer.transform(train_text)

In [23]:
test_word_features = word_vectorizer.transform(test_text)

In [24]:
char_vectorizer = TfidfVectorizer(
   sublinear_tf=True,
   strip_accents='unicode',
   analyzer='char',
   stop_words='english',
   ngram_range=(2,6),
   max_features=50000)

char_vectorizer.fit(all_text)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:539: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  "The parameter 'stop_words' will not be used"


TfidfVectorizer(analyzer='char', max_features=50000, ngram_range=(2, 6),
                stop_words='english', strip_accents='unicode',
                sublinear_tf=True)

In [25]:
train_char_features = char_vectorizer.transform(train_text)

In [27]:
test_char_features = char_vectorizer.transform(test_text)

In [1]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

NameError: ignored

In [ ]:
scores = []